# Réalisez une veille technique : 
*Notebook Mission 2 - Réalisez une veille technique*

**Classification superviée d’images via les Visions Transformers (ViT)**

Il s'agit de réaliser un POC qui met en oeuvre cette nouvelle technique avec les données image déjà exploitées dans le cadre du projet 6.

In [170]:
# fonctions personnelles pour le pré traitement des données textuelles
import pre_treatment_text as ptt

import numpy as np

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
#from matplotlib.ticker import ScalarFormatter
#from matplotlib.ticker import FuncFormatter
#import scipy
#from scipy import stats
#import scipy.stats as st

#import statsmodels
#import statsmodels.api as sm

#from collections import defaultdict
#from collections import Counter

#import cv2

import os
from os import listdir

import torch
import timm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from transformers import ViTFeatureExtractor, ViTModel
from transformers import TFViTForImageClassification, ViTImageProcessor
from transformers import TFAutoModelForImageClassification, AutoFeatureExtractor
from tensorflow.keras.preprocessing import image_dataset_from_directory
from transformers import AdamW

#import shutil

print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("matplotlib version", matplotlib.__version__)
#print("seaborn version", sns.__version__)
#print("scipy version", scipy.__version__)
#print("statsmodels version", statsmodels.__version__)

#print("sklearn version", sklearn.__version__)

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

numpy version 1.26.4
pandas version 2.1.4
matplotlib version 3.8.0


## 1 - Préparation des données

In [41]:
# Charger le fichier de données
data = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_6/data_projet/Dataset+projet+prétraitement+textes+images/Flipkart/flipkart_com-ecommerce_sample_1050.csv", \
                   sep=',', low_memory=False)
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa..."


In [43]:
# Appliquer la méthode split sur chaque élément de la colonne
y = data['product_category_tree'].apply(lambda x: ptt.clean_category_first_level(x))
data['category'] = y

In [45]:
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,category
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa...",Baby Care
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",...",Baby Care
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa...",Home Furnishing


**Créons les dataset d'entrainement, de validation et de test**

In [105]:
def dataset_fct(path, validation_split=0, data_type=None) :
    """
        path : Chemin du dossier contenant les images organisées en sous-dossiers par classe.
        validation_split : Pourcentage du dataset à utiliser pour la validation (ex. 0.2 pour 20%).
        data_type : Détermine si on charge les données d'entraînement ("training") ou de validation ("validation").
    """

    #  Création du dataset
    dataset = tf.keras.utils.image_dataset_from_directory(
                    path, labels='inferred', label_mode='categorical',
                    class_names=None, batch_size=batch_size, image_size=(224, 224), shuffle=True, seed=42,
                    validation_split=validation_split, subset=data_type
                    )
    return dataset

In [101]:
batch_size = 32
path = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category"
path_test = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category_test"

In [107]:
dataset_train = dataset_fct(path, validation_split=0.25, data_type='training')
dataset_val = dataset_fct(path, validation_split=0.25, data_type='validation')
dataset_test = dataset_fct(path_test, validation_split=0, data_type=None)

Found 787 files belonging to 7 classes.
Using 591 files for training.
Found 787 files belonging to 7 classes.
Using 196 files for validation.
Found 263 files belonging to 7 classes.


In [115]:
# Charger un batch d'exemple
for image_batch, label_batch in dataset_train.take(1):
    print(image_batch.shape)  # Afficher la forme du batch d'images

(32, 224, 224, 3)


## 2 - Classifier les images

solution pour préparer vos données pour un modèle Vision Transformer (ViT) en utilisant TensorFlow et Hugging Face.

Solution
Nous allons suivre ces étapes pour charger les images, les prétraiter, puis entraîner un modèle ViT sur les données :

Charger les données à partir des dossiers : Utiliser la fonction image_dataset_from_directory de TensorFlow pour charger les images à partir des sous-dossiers par catégorie.

Prétraitement des images : Utiliser le ViTFeatureExtractor de Hugging Face pour adapter les images aux exigences du modèle ViT.

Création du modèle ViT : Charger un modèle ViT pré-entraîné pour la classification d'images et l'adapter pour l'entraînement.

In [194]:
from transformers import TFAutoModelForImageClassification, AdamWeightDecay, AutoImageProcessor
from tensorflow.keras.optimizers import Adam

tf.config.run_functions_eagerly(True)

# Définir le chemin vers les dossiers contenant les images
train_dir = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category"
test_dir = "C:/Users/admin/Documents/Projets/Projet_6/data_projet/Flipkart/images_per_category_test"

# Chargement de l'extracteur de caractéristiques pour ViT
feature_extractor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

# Fonction de prétraitement des images pour ViT avec @tf.function pour optimiser le graph
@tf.function
def preprocess_vit(image):
    # Normalisation de l'image
    image = image / 255.0
    # Extraction des caractéristiques de l'image avec le feature_extractor
    inputs = feature_extractor(images=image, return_tensors="tf")
    return inputs['pixel_values']

# Chargement des datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode="int",  # Utiliser "int" pour une classification des labels
    shuffle=True
)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode="int",
    shuffle=False
)

# Application du prétraitement aux datasets
train_dataset = train_dataset.map(lambda x, y: (preprocess_vit(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(lambda x, y: (preprocess_vit(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Chargement du modèle ViT pré-entraîné pour la classification d'images
model = TFAutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=7)

# Compiler le modèle avec l'optimiseur AdamW et la fonction de perte SparseCategoricalCrossentropy
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Entraînement du modèle
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Évaluation du modèle sur les données de validation
loss, accuracy = model.evaluate(test_dataset)
print(f"Perte : {loss:.4f}, Précision : {accuracy:.4%}")



Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Found 787 files belonging to 7 classes.
Found 263 files belonging to 7 classes.


C:\Users\admin\anaconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\admin\AppData\Local\Temp\ipykernel_4952\2941086337.py", line 40, in None  *
        lambda x, y: (preprocess_vit(x), y)
    File "C:\Users\admin\AppData\Local\Temp\ipykernel_4952\3060344274.py", line 19, in preprocess_vit
        inputs = feature_extractor(images=image, return_tensors="tf")
    File "C:\Users\admin\anaconda3\Lib\site-packages\transformers\image_processing_utils.py", line 41, in __call__
        return self.preprocess(images, **kwargs)
    File "C:\Users\admin\anaconda3\Lib\site-packages\transformers\utils\generic.py", line 854, in wrapper
        return func(*args, **valid_kwargs)
    File "C:\Users\admin\anaconda3\Lib\site-packages\transformers\models\vit\image_processing_vit.py", line 226, in preprocess
        images = make_list_of_images(images)
    File "C:\Users\admin\anaconda3\Lib\site-packages\transformers\image_utils.py", line 196, in make_list_of_images
        images = list(images)

    Operat

In [ ]:
loss, accuracy = final_model.evaluate(val_dataset)
print(f"Perte : {loss:.4f}, Précision : {accuracy:.4%}")